In [1]:
import pandas as pd
import numpy as np

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(f'\nFirst 10 rows:')
df.head(10)

Shape: (20100, 4)

First 10 rows:


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [2]:
# Analyze the sample submission - calculate score
from decimal import Decimal, getcontext
getcontext().prec = 30

# Parse the data
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['x_val'] = df['x'].str[1:].astype(float)
df['y_val'] = df['y'].str[1:].astype(float)
df['deg_val'] = df['deg'].str[1:].astype(float)

# Calculate score for each N
scores = []
for n in range(1, 201):
    group = df[df['n'] == n]
    x_min, x_max = group['x_val'].min(), group['x_val'].max()
    y_min, y_max = group['y_val'].min(), group['y_val'].max()
    
    # Note: This is approximate - actual bounding box needs tree polygon geometry
    # For now, just show the center point spread
    width = x_max - x_min
    height = y_max - y_min
    side = max(width, height)
    score = side**2 / n
    scores.append({'n': n, 'width': width, 'height': height, 'side': side, 'score': score})

scores_df = pd.DataFrame(scores)
print(f"Total approximate score (center points only): {scores_df['score'].sum():.6f}")
print(f"\\nNote: Actual score requires tree polygon geometry (adds ~0.7-1.0 to each dimension)")
print(f"\\nTop 10 largest score contributions:")
scores_df.nlargest(10, 'score')

Total approximate score (center points only): 114.091531
\nNote: Actual score requires tree polygon geometry (adds ~0.7-1.0 to each dimension)
\nTop 10 largest score contributions:


,n,width,height,side,score
190,191,10.808311,11.434470,11.434470,0.684540
191,192,10.808311,11.434470,11.434470,0.680975
192,193,10.808311,11.434470,11.434470,0.677446
178,179,10.207242,10.985385,10.985385,0.674183
193,194,11.087639,11.434470,11.434470,0.673954
194,195,11.087639,11.434470,11.434470,0.670498
179,180,10.207242,10.985385,10.985385,0.670437
195,196,11.087639,11.434470,11.434470,0.667077
180,181,10.287624,10.985385,10.985385,0.666733
196,197,11.087639,11.434470,11.434470,0.663691


In [3]:
# Check rotation distribution
print("Rotation angle distribution:")
print(df['deg_val'].value_counts().head(20))
print(f"\\nUnique rotation angles: {df['deg_val'].nunique()}")

Rotation angle distribution:
deg_val
180.0    5908
90.0     5348
0.0      4779
270.0    4065
Name: count, dtype: int64
\nUnique rotation angles: 4
